In [1]:
import pyspark
from pyspark.sql import SparkSession
import os

In [2]:
packages = [
    "org.apache.iceberg:iceberg-spark-runtime-3.5_2.12:1.5.2",
    "org.projectnessie.nessie-integrations:nessie-spark-extensions-3.5_2.12:0.83.1",
    "org.apache.hadoop:hadoop-aws:3.3.4",
    "org.apache.spark:spark-avro_2.12:3.5.0"
]
conf = (
pyspark.SparkConf()
.setAppName('spark-iceberg-lab1')
.set('spark.jars.packages', ",".join(packages))
.set('spark.sql.extensions', 'org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions')
.set('spark.sql.catalog.nessie', 'org.apache.iceberg.spark.SparkCatalog')
.set('spark.sql.catalog.nessie.catalog-impl', 'org.apache.iceberg.nessie.NessieCatalog')
.set('spark.sql.catalog.nessie.uri', 'http://nessie:19120/api/v1')
.set('spark.sql.catalog.nessie.ref', 'main')
.set('spark.sql.catalog.nessie.authentication.type', 'NONE')
.set('spark.sql.catalog.nessie.warehouse', 's3a://warehouse/')
.set('spark.hadoop.fs.s3a.endpoint', 'http://minio:9000')
.set('spark.hadoop.fs.s3a.access.key', 'admin')
.set('spark.hadoop.fs.s3a.secret.key', 'password')
.set('spark.hadoop.fs.s3a.path.style.access', 'true')
.set('spark.hadoop.fs.s3a.connection.ssl.enabled', 'false')
)

In [3]:
spark = SparkSession.builder.config(conf=conf).getOrCreate()

:: loading settings :: url = jar:file:/home/docker/.local/lib/python3.10/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/docker/.ivy2/cache
The jars for the packages stored in: /home/docker/.ivy2/jars
org.apache.iceberg#iceberg-spark-runtime-3.5_2.12 added as a dependency
org.projectnessie.nessie-integrations#nessie-spark-extensions-3.5_2.12 added as a dependency
org.apache.hadoop#hadoop-aws added as a dependency
org.apache.spark#spark-avro_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-4513f097-3680-4036-a256-cf856e0c1c1d;1.0
	confs: [default]
	found org.apache.iceberg#iceberg-spark-runtime-3.5_2.12;1.5.2 in central
	found org.projectnessie.nessie-integrations#nessie-spark-extensions-3.5_2.12;0.83.1 in central
	found org.apache.hadoop#hadoop-aws;3.3.4 in central
	found com.amazonaws#aws-java-sdk-bundle;1.12.262 in central
	found org.wildfly.openssl#wildfly-openssl;1.0.7.Final in central
	found org.apache.spark#spark-avro_2.12;3.5.0 in central
	found org.tukaani#xz;1.9 in central
:: resolution report :: resolve 760ms :: artifac

In [5]:
# Read the Avro manifest list
df = spark.read.format("avro").load("s3a://warehouse/demo/employee_f1ae59e9-eab2-4bba-bf4a-1b2fff895d26/metadata/snap-4776718034125825857-1-a461d5ed-9ebd-4a76-88ef-83515d85d8a9.avro")

# Show it as a table (easy reading)
df.show(truncate=False)

25/12/23 18:51:56 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties


+------------------------------------------------------------------------------------------------------------------------+---------------+-----------------+-------+---------------+-------------------+-------------------+-----------------+--------------------+-------------------+----------------+-------------------+------------------+----------------------------------------+
|manifest_path                                                                                                           |manifest_length|partition_spec_id|content|sequence_number|min_sequence_number|added_snapshot_id  |added_files_count|existing_files_count|deleted_files_count|added_rows_count|existing_rows_count|deleted_rows_count|partitions                              |
+------------------------------------------------------------------------------------------------------------------------+---------------+-----------------+-------+---------------+-------------------+-------------------+-----------------+--------

In [6]:
spark.sql("""
SELECT * FROM nessie.demo.employee.snapshots
""").show(truncate=False)

+-----------------------+-------------------+---------+---------+------------------------------------------------------------------------------------------------------------------------------------------------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|committed_at           |snapshot_id        |parent_id|operation|manifest_list                                                                                                                                   |summary                                                                                                                                                                                                                                                                                         |


In [7]:
spark.sql("""
SELECT * FROM nessie.demo.employee.manifests
""").show(truncate=False)

+-------+------------------------------------------------------------------------------------------------------------------------+------+-----------------+-------------------+----------------------+-------------------------+------------------------+------------------------+---------------------------+--------------------------+--------------------------+
|content|path                                                                                                                    |length|partition_spec_id|added_snapshot_id  |added_data_files_count|existing_data_files_count|deleted_data_files_count|added_delete_files_count|existing_delete_files_count|deleted_delete_files_count|partition_summaries       |
+-------+------------------------------------------------------------------------------------------------------------------------+------+-----------------+-------------------+----------------------+-------------------------+------------------------+------------------------+------------

In [8]:
spark.sql("""
SELECT * FROM nessie.demo.employee.files
""").show(truncate=False)

+-------+------------------------------------------------------------------------------------------------------------------------------------------------+-----------+-------+---------+------------+------------------+------------------------------------------------------+------------------------------------------------+------------------------------------------------+----------------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------+------------+-------------+------------+-------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|co

In [9]:
# Read the Avro manifest list
df = spark.read.format("avro").load("s3a://warehouse/demo/employee_f1ae59e9-eab2-4bba-bf4a-1b2fff895d26/metadata/a461d5ed-9ebd-4a76-88ef-83515d85d8a9-m0.avro")

# Show it as a table (easy reading)
df.show(truncate=False)

+------+-------------------+---------------+--------------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|status|snapshot_id        |sequence_number|file_sequence_number|data_file                                                                                                                                                            

In [10]:
spark.sql("""
show tables in nessie.demo
""").show()

+---------+--------------------+-----------+
|namespace|           tableName|isTemporary|
+---------+--------------------+-----------+
|     demo|     emp_partitioned|      false|
|     demo|emp_partitioned_m...|      false|
|     demo|            employee|      false|
+---------+--------------------+-----------+



In [11]:
spark.sql("""
select * from nessie.demo.employee
""").show()

+---+-----------------+-----------+-------+------+-------+
| id|            title| department| salary|region|manager|
+---+-----------------+-----------+-------+------+-------+
|  2|         Director|      Sales|22000.0|  EMEA|   Bo W|
|  1|Software Engineer|Engineering|25000.0|    NA|   Jack|
+---+-----------------+-----------+-------+------+-------+



In [16]:
spark.sql("""
create table nessie.demo.employee_updates (
   id int,
   title string,
   department string,
   salary float,
   region string,
   manager string )
using iceberg
""")

DataFrame[]

In [14]:
spark.sql("""
select * from nessie.demo.employee
""").show()

+---+-----------------+-----------+-------+------+-------+
| id|            title| department| salary|region|manager|
+---+-----------------+-----------+-------+------+-------+
|  2|         Director|      Sales|22000.0|  EMEA|   Bo W|
|  1|Software Engineer|Engineering|25000.0|    NA|   Jack|
+---+-----------------+-----------+-------+------+-------+



In [17]:
spark.sql("""
show tables in nessie.demo
""").show()

+---------+--------------------+-----------+
|namespace|           tableName|isTemporary|
+---------+--------------------+-----------+
|     demo|     emp_partitioned|      false|
|     demo|emp_partitioned_m...|      false|
|     demo|            employee|      false|
|     demo|    employee_updates|      false|
+---------+--------------------+-----------+



In [18]:
spark.sql("INSERT INTO nessie.demo.employee_updates VALUES (3, 'Manager', 'Engineering', 25000, 'APAC', 'Bo W'), (4, 'Developer', 'Engineering', 12000, 'EMEA', 'Bo W')")

DataFrame[]

In [19]:
spark.sql("""
select * from nessie.demo.employee_updates
""").show()

+---+---------+-----------+-------+------+-------+
| id|    title| department| salary|region|manager|
+---+---------+-----------+-------+------+-------+
|  3|  Manager|Engineering|25000.0|  APAC|   Bo W|
|  4|Developer|Engineering|12000.0|  EMEA|   Bo W|
+---+---------+-----------+-------+------+-------+



In [21]:
spark.sql("""
merge into nessie.demo.employee as target
using (select * from nessie.demo.employee_updates) as source
on source.id = target.id
when matched and source.title = 'Manager' and source.salary > 10000 then
   update set target.salary = source.salary
when not matched then
   insert *
""")  

DataFrame[]

In [22]:
spark.sql("""
select * from nessie.demo.employee
""").show()

+---+-----------------+-----------+-------+------+-------+
| id|            title| department| salary|region|manager|
+---+-----------------+-----------+-------+------+-------+
|  3|          Manager|Engineering|25000.0|  APAC|   Bo W|
|  2|         Director|      Sales|22000.0|  EMEA|   Bo W|
|  4|        Developer|Engineering|12000.0|  EMEA|   Bo W|
|  1|Software Engineer|Engineering|25000.0|    NA|   Jack|
+---+-----------------+-----------+-------+------+-------+



In [23]:
# Read the Avro manifest list
df = spark.read.format("avro").load("s3a://warehouse/demo/employee_f1ae59e9-eab2-4bba-bf4a-1b2fff895d26/metadata/snap-5620577962586266168-1-d7016d66-b1e4-439b-a800-eb7a2e35f080.avro")

# Show it as a table (easy reading)
df.show(truncate=False)

+------------------------------------------------------------------------------------------------------------------------+---------------+-----------------+-------+---------------+-------------------+-------------------+-----------------+--------------------+-------------------+----------------+-------------------+------------------+----------------------------------------------+
|manifest_path                                                                                                           |manifest_length|partition_spec_id|content|sequence_number|min_sequence_number|added_snapshot_id  |added_files_count|existing_files_count|deleted_files_count|added_rows_count|existing_rows_count|deleted_rows_count|partitions                                    |
+------------------------------------------------------------------------------------------------------------------------+---------------+-----------------+-------+---------------+-------------------+-------------------+--------------

In [24]:
# Read the Avro manifest list
df = spark.read.format("avro").load("s3a://warehouse/demo/employee_f1ae59e9-eab2-4bba-bf4a-1b2fff895d26/metadata/d7016d66-b1e4-439b-a800-eb7a2e35f080-m0.avro")

# Show it as a table (easy reading)
df.show(truncate=False)

+------+-------------------+---------------+--------------------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|status|snapshot_id        |sequence_number|file_sequence_number|data_file                                                                                                                                                                                     

In [25]:
spark.sql("""
SELECT * FROM nessie.demo.employee.files
""").show(truncate=False)

+-------+-------------------------------------------------------------------------------------------------------------------------------------------------+-----------+-------+---------+------------+------------------+------------------------------------------------------+------------------------------------------------+------------------------------------------------+----------------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------+------------+-------------+------------+-------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|c

In [26]:
spark.sql("""
CREATE TABLE nessie.demo.employee_source
USING iceberg
PARTITIONED BY (region)
AS SELECT *
FROM nessie.demo.employee
""")

DataFrame[]

In [27]:
spark.sql("""
select * from nessie.demo.employee_source
""").show()

+---+-----------------+-----------+-------+------+-------+
| id|            title| department| salary|region|manager|
+---+-----------------+-----------+-------+------+-------+
|  3|          Manager|Engineering|25000.0|  APAC|   Bo W|
|  1|Software Engineer|Engineering|25000.0|    NA|   Jack|
|  4|        Developer|Engineering|12000.0|  EMEA|   Bo W|
|  2|         Director|      Sales|22000.0|  EMEA|   Bo W|
+---+-----------------+-----------+-------+------+-------+



In [28]:
spark.sql("""
UPDATE nessie.demo.employee_source
SET title = 'Manager', salary = 25000
WHERE id = 2
""")

DataFrame[]

In [29]:
spark.sql("""
select * from nessie.demo.employee_source
""").show()

+---+-----------------+-----------+-------+------+-------+
| id|            title| department| salary|region|manager|
+---+-----------------+-----------+-------+------+-------+
|  4|        Developer|Engineering|12000.0|  EMEA|   Bo W|
|  2|          Manager|      Sales|25000.0|  EMEA|   Bo W|
|  3|          Manager|Engineering|25000.0|  APAC|   Bo W|
|  1|Software Engineer|Engineering|25000.0|    NA|   Jack|
+---+-----------------+-----------+-------+------+-------+



In [30]:
spark.sql("""
select * from nessie.demo.employee_source.snapshots
""").show()

+--------------------+-------------------+-------------------+---------+--------------------+--------------------+
|        committed_at|        snapshot_id|          parent_id|operation|       manifest_list|             summary|
+--------------------+-------------------+-------------------+---------+--------------------+--------------------+
|2025-12-23 22:08:...|6834224639275070960|               NULL|   append|s3a://warehouse/d...|{spark.app.id -> ...|
|2025-12-23 22:19:...|6829033160407099953|6834224639275070960|overwrite|s3a://warehouse/d...|{spark.app.id -> ...|
+--------------------+-------------------+-------------------+---------+--------------------+--------------------+



In [32]:
# Read the Avro manifest list
df = spark.read.format("avro").load("s3a://warehouse/demo/employee_source_b73892fc-14a1-4e25-bb9b-323d038dc25d/metadata/snap-6834224639275070960-1-fb2f3cfd-ff53-4650-afc2-a2d7f31a8ea8.avro")

# Show it as a table (easy reading)
df.show(truncate=False)

+-------------------------------------------------------------------------------------------------------------------------------+---------------+-----------------+-------+---------------+-------------------+-------------------+-----------------+--------------------+-------------------+----------------+-------------------+------------------+----------------------------------------+
|manifest_path                                                                                                                  |manifest_length|partition_spec_id|content|sequence_number|min_sequence_number|added_snapshot_id  |added_files_count|existing_files_count|deleted_files_count|added_rows_count|existing_rows_count|deleted_rows_count|partitions                              |
+-------------------------------------------------------------------------------------------------------------------------------+---------------+-----------------+-------+---------------+-------------------+-------------------+-----

In [33]:
# Read the Avro manifest list
df = spark.read.format("avro").load("s3a://warehouse/demo/employee_source_b73892fc-14a1-4e25-bb9b-323d038dc25d/metadata/fb2f3cfd-ff53-4650-afc2-a2d7f31a8ea8-m0.avro")

# Show it as a table (easy reading)
df.show(truncate=False)

+------+-------------------+---------------+--------------------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|status|snapshot_id        |sequence_number|file_sequence_number|data_file                                                                                                                                                    

In [36]:
spark.sql("""
select * from nessie.demo.employee_source.manifests
""").show(truncate=False)

+-------+-------------------------------------------------------------------------------------------------------------------------------+------+-----------------+-------------------+----------------------+-------------------------+------------------------+------------------------+---------------------------+--------------------------+----------------------------+
|content|path                                                                                                                           |length|partition_spec_id|added_snapshot_id  |added_data_files_count|existing_data_files_count|deleted_data_files_count|added_delete_files_count|existing_delete_files_count|deleted_delete_files_count|partition_summaries         |
+-------+-------------------------------------------------------------------------------------------------------------------------------+------+-----------------+-------------------+----------------------+-------------------------+------------------------+------------

In [38]:
spark.sql("DELETE FROM nessie.demo.employee WHERE id < 3")

DataFrame[]

In [4]:
spark.sql("""
select * from nessie.demo.employee
""").show()

25/12/29 21:44:47 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties


+---+---------+-----------+-------+------+-------+
| id|    title| department| salary|region|manager|
+---+---------+-----------+-------+------+-------+
|  3|  Manager|Engineering|25000.0|  APAC|   Bo W|
|  4|Developer|Engineering|12000.0|  EMEA|   Bo W|
+---+---------+-----------+-------+------+-------+



In [41]:
spark.sql("""SELECT * FROM nessie.demo.employee.metadata_log_entries
""").show(truncate=False)

+-----------------------+------------------------------------------------------------------------------------------------------------------------------------+-------------------+----------------+----------------------+
|timestamp              |file                                                                                                                                |latest_snapshot_id |latest_schema_id|latest_sequence_number|
+-----------------------+------------------------------------------------------------------------------------------------------------------------------------+-------------------+----------------+----------------------+
|2025-12-20 19:06:28.317|s3a://warehouse/demo/employee_f1ae59e9-eab2-4bba-bf4a-1b2fff895d26/metadata/00000-d636e9e1-f73b-4c55-9712-132f8c4e7d8d.metadata.json|NULL               |NULL            |NULL                  |
|2025-12-20 19:26:43.429|s3a://warehouse/demo/employee_f1ae59e9-eab2-4bba-bf4a-1b2fff895d26/metadata/00001-3efbdb2d-34de-4d2

In [42]:
spark.sql("""SELECT * FROM nessie.demo.employee.snapshots
""").show(truncate=False)

+-----------------------+-------------------+-------------------+---------+------------------------------------------------------------------------------------------------------------------------------------------------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|committed_at           |snapshot_id        |parent_id          |operation|manifest_list                                                                                                                                   |summary                                                                                                                                                                                                                                                                 

In [5]:
spark.sql("""
desc formatted nessie.demo.employee_source
""").show()

+--------------------+--------------------+-------+
|            col_name|           data_type|comment|
+--------------------+--------------------+-------+
|                  id|                 int|   NULL|
|               title|              string|   NULL|
|          department|              string|   NULL|
|              salary|               float|   NULL|
|              region|              string|   NULL|
|             manager|              string|   NULL|
|# Partition Infor...|                    |       |
|          # col_name|           data_type|comment|
|              region|              string|   NULL|
|                    |                    |       |
|  # Metadata Columns|                    |       |
|            _spec_id|                 int|       |
|          _partition|struct<region:str...|       |
|               _file|              string|       |
|                _pos|              bigint|       |
|            _deleted|             boolean|       |
|           

In [6]:
spark.sql("""
desc formatted nessie.demo.employee
""").show()

+--------------------+--------------------+-------+
|            col_name|           data_type|comment|
+--------------------+--------------------+-------+
|                  id|                 int|   NULL|
|               title|              string|   NULL|
|          department|              string|   NULL|
|              salary|               float|   NULL|
|              region|              string|   NULL|
|             manager|              string|   NULL|
|# Partition Infor...|                    |       |
|          # col_name|           data_type|comment|
|              region|              string|   NULL|
|                    |                    |       |
|  # Metadata Columns|                    |       |
|            _spec_id|                 int|       |
|          _partition|struct<region:str...|       |
|               _file|              string|       |
|                _pos|              bigint|       |
|            _deleted|             boolean|       |
|           

In [10]:
spark.sql("""
  SELECT * FROM nessie.demo.employee.partitions
""").show()

+---------+-------+------------+----------+-----------------------------+----------------------------+--------------------------+----------------------------+--------------------------+--------------------+------------------------+
|partition|spec_id|record_count|file_count|total_data_file_size_in_bytes|position_delete_record_count|position_delete_file_count|equality_delete_record_count|equality_delete_file_count|     last_updated_at|last_updated_snapshot_id|
+---------+-------+------------+----------+-----------------------------+----------------------------+--------------------------+----------------------------+--------------------------+--------------------+------------------------+
|   {APAC}|      1|           1|         1|                         1680|                           0|                         0|                           0|                         0|2025-12-23 21:17:...|     5620577962586266168|
|   {EMEA}|      1|           1|         1|                         1694

In [11]:
spark.sql("""
  SELECT * FROM nessie.demo.employee
""").show()

+---+---------+-----------+-------+------+-------+
| id|    title| department| salary|region|manager|
+---+---------+-----------+-------+------+-------+
|  3|  Manager|Engineering|25000.0|  APAC|   Bo W|
|  4|Developer|Engineering|12000.0|  EMEA|   Bo W|
+---+---------+-----------+-------+------+-------+



In [13]:
spark.sql("""
  SELECT * FROM nessie.demo.employee_source
""").show()

+---+-----------------+-----------+-------+------+-------+
| id|            title| department| salary|region|manager|
+---+-----------------+-----------+-------+------+-------+
|  3|          Manager|Engineering|25000.0|  APAC|   Bo W|
|  4|        Developer|Engineering|12000.0|  EMEA|   Bo W|
|  2|          Manager|      Sales|25000.0|  EMEA|   Bo W|
|  1|Software Engineer|Engineering|25000.0|    NA|   Jack|
+---+-----------------+-----------+-------+------+-------+



In [15]:
spark.sql("""
select * from nessie.demo.employee_source
where region = 'EMEA'
""").show()

+---+---------+-----------+-------+------+-------+
| id|    title| department| salary|region|manager|
+---+---------+-----------+-------+------+-------+
|  4|Developer|Engineering|12000.0|  EMEA|   Bo W|
|  2|  Manager|      Sales|25000.0|  EMEA|   Bo W|
+---+---------+-----------+-------+------+-------+



In [18]:
spark.sql("""
insert overwrite nessie.demo.employee
partition(region = 'EMEA')
select id, title, department, salary, manager from nessie.demo.employee_source
where region = 'EMEA'
""")

DataFrame[]

In [19]:
spark.sql("""
  SELECT * FROM nessie.demo.employee
""").show()

+---+---------+-----------+-------+------+-------+
| id|    title| department| salary|region|manager|
+---+---------+-----------+-------+------+-------+
|  3|  Manager|Engineering|25000.0|  APAC|   Bo W|
|  2|  Manager|      Sales|25000.0|  EMEA|   Bo W|
|  4|Developer|Engineering|12000.0|  EMEA|   Bo W|
+---+---------+-----------+-------+------+-------+



In [21]:
spark.sql("""
  SELECT * FROM nessie.demo.employee.manifests
""").show(truncate=False)

+-------+------------------------------------------------------------------------------------------------------------------------+------+-----------------+-------------------+----------------------+-------------------------+------------------------+------------------------+---------------------------+--------------------------+----------------------------+
|content|path                                                                                                                    |length|partition_spec_id|added_snapshot_id  |added_data_files_count|existing_data_files_count|deleted_data_files_count|added_delete_files_count|existing_delete_files_count|deleted_delete_files_count|partition_summaries         |
+-------+------------------------------------------------------------------------------------------------------------------------+------+-----------------+-------------------+----------------------+-------------------------+------------------------+------------------------+--------

In [22]:
spark.sql("""
  SELECT * FROM nessie.demo.employee.files
""").show(truncate=False)

+-------+-------------------------------------------------------------------------------------------------------------------------------------------------+-----------+-------+---------+------------+------------------+------------------------------------------------------+------------------------------------------------+------------------------------------------------+----------------+------------------------------------------------------------------------------------------------------------------------------------------------------------+------------------------------------------------------------------------------------------------------------------------------------------------------+------------+-------------+------------+-------------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|content|file_path                                  

In [23]:
spark.conf.set("spark.sql.sources.partitionOverwriteMode", "dynamic")

In [24]:
spark.sql("""
INSERT OVERWRITE nessie.demo.employee
SELECT * FROM nessie.demo.employee_source
WHERE region = 'NA'
""")

DataFrame[]

In [25]:
spark.sql("""
  SELECT * FROM nessie.demo.employee
""").show()

+---+-----------------+-----------+-------+------+-------+
| id|            title| department| salary|region|manager|
+---+-----------------+-----------+-------+------+-------+
|  1|Software Engineer|Engineering|25000.0|    NA|   Jack|
|  2|          Manager|      Sales|25000.0|  EMEA|   Bo W|
|  4|        Developer|Engineering|12000.0|  EMEA|   Bo W|
|  3|          Manager|Engineering|25000.0|  APAC|   Bo W|
+---+-----------------+-----------+-------+------+-------+



In [26]:
spark.sql("""
  SELECT * FROM nessie.demo.employee.files
""").show(truncate=False)

+-------+-------------------------------------------------------------------------------------------------------------------------------------------------+-----------+-------+---------+------------+------------------+------------------------------------------------------+------------------------------------------------+------------------------------------------------+----------------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------+------------+-------------+------------+-------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|c

In [28]:
spark.sql("DELETE FROM nessie.demo.employee WHERE id = 2")

DataFrame[]

In [29]:
spark.sql("""
  SELECT * FROM nessie.demo.employee
""").show()

+---+-----------------+-----------+-------+------+-------+
| id|            title| department| salary|region|manager|
+---+-----------------+-----------+-------+------+-------+
|  1|Software Engineer|Engineering|25000.0|    NA|   Jack|
|  4|        Developer|Engineering|12000.0|  EMEA|   Bo W|
|  3|          Manager|Engineering|25000.0|  APAC|   Bo W|
+---+-----------------+-----------+-------+------+-------+



In [30]:
spark.sql("""
  SELECT * FROM nessie.demo.employee.files
""").show(truncate=False)

+-------+-------------------------------------------------------------------------------------------------------------------------------------------------+-----------+-------+---------+------------+------------------+------------------------------------------------------+------------------------------------------------+------------------------------------------------+----------------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------+------------+-------------+------------+-------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|c

In [40]:
spark.sql("ALTER TABLE nessie.demo.employee SET TBLPROPERTIES ('gc.enabled' = 'true')")
spark.sql(f"""
    CALL nessie.system.expire_snapshots(
        table => 'nessie.demo.employee', 
        older_than => TIMESTAMP '2025-12-24 00:00:00', 
        retain_last => 4
    )
""")

DataFrame[deleted_data_files_count: bigint, deleted_position_delete_files_count: bigint, deleted_equality_delete_files_count: bigint, deleted_manifest_files_count: bigint, deleted_manifest_lists_count: bigint, deleted_statistics_files_count: bigint]

In [42]:
spark.sql("CALL nessie.system.rewrite_data_files('demo.employee')").show()

+--------------------------+----------------------+---------------------+-----------------------+
|rewritten_data_files_count|added_data_files_count|rewritten_bytes_count|failed_data_files_count|
+--------------------------+----------------------+---------------------+-----------------------+
|                         0|                     0|                    0|                      0|
+--------------------------+----------------------+---------------------+-----------------------+



In [44]:
spark.sql("""
CALL nessie.system.rewrite_data_files(
  table => 'demo.employee',
  strategy => 'sort',
  sort_order => 'id ASC' -- You can also use 'id DESC, manager ASC'
)
""")

DataFrame[rewritten_data_files_count: int, added_data_files_count: int, rewritten_bytes_count: bigint, failed_data_files_count: int]

In [45]:
spark.sql("""
  SELECT * FROM nessie.demo.employee
""").show()

+---+-----------------+-----------+-------+------+-------+
| id|            title| department| salary|region|manager|
+---+-----------------+-----------+-------+------+-------+
|  3|          Manager|Engineering|25000.0|  APAC|   Bo W|
|  1|Software Engineer|Engineering|25000.0|    NA|   Jack|
|  4|        Developer|Engineering|12000.0|  EMEA|   Bo W|
+---+-----------------+-----------+-------+------+-------+



In [48]:
spark.sql("""
show create table nessie.demo.employee
""").show(truncate=False)

+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|createtab_stmt                                                                                                                                                                                                                              

In [51]:
spark.sql("""
desc formatted nessie.demo.employee
""").show(n=100, truncate=False)

+----------------------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-------+
|col_name                    |data_type                                                                                                                                                                                                                                                                                                                                                                             |comment|
+----------------------------+----------------------------------------------------------------------------------------------------------------------------------------------

In [52]:
spark.sql("""
desc formatted nessie.demo.employee
""").toPandas()

ImportError: Pandas >= 1.0.5 must be installed; however, it was not found.

In [54]:
spark.sql("""
ALTER TABLE nessie.demo.employee SET TBLPROPERTIES (
'write.delete.mode'='merge-on-read',
'write.update.mode'='copy-on-write',
'write.merge.mode'='copy-on-write'
)
""")

DataFrame[]

In [55]:
spark.sql("""
update nessie.demo.employee
set salary = 20000
where id = 1
""")

DataFrame[]

In [56]:
spark.sql("""
  SELECT * FROM nessie.demo.employee
""").show()

+---+-----------------+-----------+-------+------+-------+
| id|            title| department| salary|region|manager|
+---+-----------------+-----------+-------+------+-------+
|  3|          Manager|Engineering|25000.0|  APAC|   Bo W|
|  4|        Developer|Engineering|12000.0|  EMEA|   Bo W|
|  1|Software Engineer|Engineering|20000.0|    NA|   Jack|
+---+-----------------+-----------+-------+------+-------+



In [57]:
spark.sql("""
  SELECT * FROM nessie.demo.employee.files
""").show(truncate=False)

+-------+-------------------------------------------------------------------------------------------------------------------------------------------------+-----------+-------+---------+------------+------------------+------------------------------------------------------+------------------------------------------------+------------------------------------------------+----------------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------+------------+-------------+------------+-------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|c

In [58]:
spark.sql("""
  SELECT * FROM nessie.demo.employee.manifests
""").show(truncate=False)

+-------+------------------------------------------------------------------------------------------------------------------------+------+-----------------+-------------------+----------------------+-------------------------+------------------------+------------------------+---------------------------+--------------------------+----------------------------+
|content|path                                                                                                                    |length|partition_spec_id|added_snapshot_id  |added_data_files_count|existing_data_files_count|deleted_data_files_count|added_delete_files_count|existing_delete_files_count|deleted_delete_files_count|partition_summaries         |
+-------+------------------------------------------------------------------------------------------------------------------------+------+-----------------+-------------------+----------------------+-------------------------+------------------------+------------------------+--------

In [59]:
# Read the Avro manifest list
df = spark.read.format("avro").load("s3a://warehouse/demo/employee_f1ae59e9-eab2-4bba-bf4a-1b2fff895d26/metadata/b7cc65f4-c922-4673-82d1-fb6209064412-m0.avro")

# Show it as a table (easy reading)
df.show(truncate=False)

+------+-------------------+---------------+--------------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|status|snapshot_id        |sequence_number|file_sequence_number|data_file                                                                                                                                                           

In [4]:
df = spark.read.parquet("s3a://warehouse/demo/employee_f1ae59e9-eab2-4bba-bf4a-1b2fff895d26/data/region=NA/00000-14-92615ee2-4abc-46b2-aee5-97fe12c66ecf-0-00001.parquet")
df.show(truncate=False)

26/01/01 01:10:44 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties


+---+-----------------+-----------+-------+------+-------+
|id |title            |department |salary |region|manager|
+---+-----------------+-----------+-------+------+-------+
|1  |Software Engineer|Engineering|25000.0|NA    |Jack   |
+---+-----------------+-----------+-------+------+-------+

